In [1]:
!pip install ccxt pandas

In [2]:
import ccxt
import pandas as pd
from datetime import datetime

# --- PART 1: Automated Data Acquisition & Extremum Detection ---
def get_crash_params():
    """
    Automated forensic data retrieval from Binance API.
    Instead of hardcoded thresholds, this function scans for the 
    Maximum Intra-day Drawdown to capture the exact flash crash moment.
    """
    exchange = ccxt.binance()
    # Target date: Dec 24, 2025 (UTC)
    since = int(datetime(2025, 12, 24, 0, 0).timestamp() * 1000)
    
    print("Scanning Binance BTC/USD1 for maximum volatility on Dec 24...")
    
    # Fetch 24 hours of 1-minute OHLCV data
    ohlcv = exchange.fetch_ohlcv('BTC/USD1', timeframe='1m', since=since, limit=1440)
    df = pd.DataFrame(ohlcv, columns=['ts', 'open', 'high', 'low', 'close', 'vol'])
    
    if df.empty:
        # Fallback: Empirical data from forensic screenshots if API returns empty
        print("Warning: API returned empty dataset. Using empirical evidence from screenshots.")
        return 87400.61, 24111.22

    # Calculate percentage drop for every minute: (Low - Open) / Open
    df['drawdown'] = (df['low'] - df['open']) / df['open']
    
    # Identify the row with the most severe price collapse
    crash_row = df.sort_values(by='drawdown').iloc[0]
    
    print(f"✅ Anomaly Detected at {datetime.fromtimestamp(crash_row['ts']/1000)}")
    print(f"Max Drawdown: {crash_row['drawdown']:.2%}")
    
    return crash_row['open'], crash_row['low']

# --- PART 2: Risk Engine & Stress Test Logic ---
def run_stress_test(market_p, crash_p, collateral_usd=1000000, debt_usd=800000):
    """
    Simulates Aave V3 Health Factor (HF) under the detected market shock.
    Formula: HF = (Collateral * (1 + drop) * Liquidation_Threshold) / Debt
    """
    drop_pct = (crash_p - market_p) / market_p
    liquidation_threshold = 0.85  # Standard Aave V3 risk parameter for WBTC
    
    # Quantifying the 'Crash Health Factor' (crash_hf)
    crash_hf = (collateral_usd * (1 + drop_pct) * liquidation_threshold) / debt_usd
    
    print(f"\n--- Stress Test Report for ${collateral_usd/1e6}M Position ---")
    print(f"Simulated Drop: {drop_pct:.2%} (From ${market_p:,.2f} to ${crash_p:,.2f})")
    print(f"Resulting Health Factor: {crash_hf:.2f}")
    
    if crash_hf < 1.0:
        # If HF < 1.0, the position is eligible for liquidation
        print(">>> STATUS: LIQUIDATION TRIGGERED.") 
    else:
        print(">>> STATUS: POSITION SAFE (Oracle Protection Active)")

# --- PART 3: Execution ---
m_price, c_price = get_crash_params()
run_stress_test(m_price, c_price)

Scanning Binance BTC/USD1 for maximum volatility on Dec 24...
✅ Anomaly Detected at 2025-12-24 15:56:00
Max Drawdown: -0.47%

--- Stress Test Report for $1.0M Position ---
Simulated Drop: -0.47% (From $86,907.38 to $86,500.02)
Resulting Health Factor: 1.06
>>> STATUS: POSITION SAFE (Oracle Protection Active)
